In [1]:
#################################################################
###################       get_p_value      ######################
#################################################################

import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from scipy.stats import poisson, norm, kstest
import numdifftools
from numpy.linalg import inv


def get_p_value(ydata, binvals, npar, make_plot, mask=[], verbose=1, yerr=None, return_teststat = False):
    
    ydata = np.array(ydata)
    #Assume poisson is gaussian with N+1 variance
    if not yerr:
        yerr = np.sqrt(ydata+1)
    else:
        yerr = np.array(yerr)

    if (npar == 3):
        def fit_func(x,p1,p2,p3):
            #see the ATLAS diboson resonance search: https://arxiv.org/pdf/1708.04445.pdf.
            xi = 0.
            y = x/13000.
            return p1*(1.-y)**(p2-xi*p3)*y**-p3
    
    elif (npar == 4):
        def fit_func(x,p1,p2,p3,p4):
            # see https://cds.cern.ch/record/2256663/files/B2G-17-001-pas.pdf
            # see the ATLAS dijet resonance search: https://arxiv.org/pdf/1806.00843.pdf.
            y = x/13000.
       
            return (p1*(1.-y)**p2) / (y**(p3 + p4*np.log(y)))
    
    elif (npar == 5):
        def fit_func(x,p1,p2,p3,p4,p5):
            # see https://cds.cern.ch/record/2256663/files/B2G-17-001-pas.pdf
            # see the ATLAS dijet resonance search: https://arxiv.org/pdf/1806.00843.pdf.
            y = x/13000.
        
            return (p1*(1.-y)**p2) / (y**(p3 + p4*np.log(y) + p5*(np.log(y))**2))

    else:
        print('Wrong number of parameters for the fit function')
    
    
    xdata = np.array([0.5*(binvals[i]+binvals[i+1]) for i in range(0,len(binvals)-1)])
    xwidths = np.array([-binvals[i]+binvals[i+1] for i in range(0,len(binvals)-1)])

    
    #Assuming inputs are bin counts, this is needed to get densities. Important for variable-width bins
    ydata = np.array(ydata) * 100 / xwidths
    yerr = np.array(yerr)*100/ np.array(xwidths)

    #Least square fit, masking out the signal region
    if len(mask) > 0:
        try:
            limits = (10**-50, np.inf)
            popt, pcov = curve_fit(fit_func, np.delete(xdata,mask), np.delete(ydata,mask), bounds=limits,
                                   sigma=np.delete(yerr,mask),max_nfev=1000000,absolute_sigma=True)
        except RuntimeError:
            print("Error - curve_fit failed")
            return 1
    else:
        try:
            popt, pcov = curve_fit(fit_func, xdata, ydata, sigma=yerr,maxfev=1000000)
        except RuntimeError:
            print("Error - curve_fit failed")
            return 1
    if verbose:
        print('fit params: ', popt)
        print('\n')
        
        
    if (npar == 3):
        ydata_fit = np.array([fit_func(x,popt[0],popt[1],popt[2]) for x in xdata])
    elif (npar == 4):
        ydata_fit = np.array([fit_func(x,popt[0],popt[1],popt[2],popt[3]) for x in xdata])
    elif (npar == 5):
        ydata_fit = np.array([fit_func(x,popt[0],popt[1],popt[2],popt[3],popt[4]) for x in xdata])
    else:
        print('Wrong number of parameters for the fit function')
    
    # Sanity check
    if len(mask) > 0:
        residuals = np.delete((ydata - ydata_fit)/yerr,mask)
    else:
        residuals = np.delete((ydata - ydata_fit)/yerr,mask)
    distance, p_val = kstest(residuals, norm(loc=0,scale=1).cdf)
    
    if (p_val < 0.1):
        print('Warning: p-value for bg only fit is p0 = %.3f' % p_val)                 
                             
    
    #Check that the function is a good fit to the sideband
    
    if verbose > 0:
        if len(mask) > 0:
            residuals = np.delete((ydata - ydata_fit)/yerr,mask)
        else:
            residuals = np.delete((ydata - ydata_fit)/yerr,mask)
        print("Goodness: ", kstest(residuals, norm(loc=0,scale=1).cdf))
        print('\n')
    
    if len(mask) == 0:
        pass
    
    # The following code is used to get the bin errors by propagating the errors on the fit params

    if (npar == 3):
        def fit_func_array(parr):
            #see the ATLAS diboson resonance search: https://arxiv.org/pdf/1708.04445.pdf.
            p1, p2, p3 = parr
            xi = 0.
            return np.array([p1*(1.-(x/13000.))**(p2-xi*p3)*(x/13000.)**-p3 for x in xdata])
    
    elif (npar == 4):
        def fit_func_array(parr):
            # see https://cds.cern.ch/record/2256663/files/B2G-17-001-pas.pdf
            # see the CMS dijet resonance search: https://arxiv.org/pdf/1806.00843.pdf.
            p1, p2, p3, p4 = parr
        
            return np.array([(p1*(1.-(x/13000.))**p2) / ((x/13000.)**(p3 + p4*np.log(x/13000.))) for x in xdata])  
    
    elif (npar == 5):
        def fit_func_array(parr):
            # see https://cds.cern.ch/record/2256663/files/B2G-17-001-pas.pdf
            # see the ATLAS dijet resonance search: https://arxiv.org/pdf/1806.00843.pdf.
            p1, p2, p3, p4, p5 = parr
        
            return np.array([(p1*(1.-(x/13000.))**p2) / ((x/13000.)**(p3 + p4*np.log(x/13000.) + p5*(np.log(x/13000.))**2)) for x in xdata])  
    
    else:
        print('Wrong number of parameters for the fit function')
    
    
    jac=numdifftools.core.Jacobian(fit_func_array)
    x_cov=np.dot(np.dot(jac(popt),pcov),jac(popt).T)
    #For plot, take systematic error band as the diagonal of the covariance matrix
    y_unc=np.sqrt([row[i] for i, row in enumerate(x_cov)])
    #print(y_unc)
    
    if (make_plot == True):
        plt.fill_between(xdata,ydata_fit+y_unc,ydata_fit-y_unc,color='gray',alpha=0.4)
        plt.hist(signal_test[:,15], bins=bins, facecolor='y', alpha=0.2, label='signal')
        #plt.errorbar(xdata, ydata,yerr,None, 'bo', label='data',markersize=3)
#        plt.errorbar(np.delete(xdata, [mpeak_ind-5-start, mpeak_ind-4-start, mpeak_ind-3-start, mpeak_ind-2-start, mpeak_ind-1-start, mpeak_ind-start, mpeak_ind+1-start, mpeak_ind+2-start, mpeak_ind+3-start, mpeak_ind+4-start, mpeak_ind+5-start, mpeak_ind+6-start]), 
#                     np.delete(ydata, [mpeak_ind-5-start, mpeak_ind-4-start, mpeak_ind-3-start, mpeak_ind-2-start, mpeak_ind-1-start, mpeak_ind-start, mpeak_ind+1-start, mpeak_ind+2-start, mpeak_ind+3-start, mpeak_ind+4-start, mpeak_ind+5-start, mpeak_ind+6-start]),
#                     np.delete(yerr, [mpeak_ind-5-start, mpeak_ind-4-start, mpeak_ind-3-start, mpeak_ind-2-start, mpeak_ind-1-start, mpeak_ind-start, mpeak_ind+1-start, mpeak_ind+2-start, mpeak_ind+3-start, mpeak_ind+4-start, mpeak_ind+5-start, mpeak_ind+6-start]),
#                     None, 'bo', label='data', markersize=4, marker='o')
        plt.errorbar(np.delete(xdata, [mpeak_ind-1-start, mpeak_ind-start, mpeak_ind+1-start]), 
                     np.delete(ydata, [mpeak_ind-1-start, mpeak_ind-start, mpeak_ind+1-start]),
                     np.delete(yerr, [mpeak_ind-1-start, mpeak_ind-start, mpeak_ind+1-start]),
                     None, 'bo', label='data', markersize=4, marker='o')
        plt.errorbar(xdata[mpeak_ind-1-start:mpeak_ind+2-start], 
                     ydata[mpeak_ind-1-start:mpeak_ind+2-start],
                     yerr[mpeak_ind-1-start:mpeak_ind+2-start], 
                     None, 'bo', label='data', markersize=4, marker='x')
        
        plt.plot(xdata, ydata_fit, 'r--', label='data')
        plt.semilogy()
        plt.ylabel(r'Num events / $160 \; \mathrm{GeV}$')
        plt.xlabel(r'$m_{JJ} \; \mathrm{GeV}$')
        plt.tick_params(which='both', direction='out', bottom=True, left=True, right=True)
        plt.minorticks_on()
        plt.ylim(ymin=0.8)

        
    if (npar == 3):
        def signal_fit_func_array(parr):
            #see the ATLAS diboson resonance search: https://arxiv.org/pdf/1708.04445.pdf.
            p1, p2, p3 = parr
            xi = 0.
            return np.array([np.sum([p1*(1.-(x/13000.))**(p2-xi*p3)*(x/13000.)**-p3*xwidths[mask[i]]/100 for i, x in enumerate(xdata[mask])])]) 
        
    elif (npar == 4):
        def signal_fit_func_array(parr):
            # see https://cds.cern.ch/record/2256663/files/B2G-17-001-pas.pdf
            # see the ATLAS dijet resonance search: https://arxiv.org/pdf/1806.00843.pdf.
            p1, p2, p3, p4 = parr
            return np.array([np.sum([(p1*(1.-(x/13000.))**p2) / ((x/13000.)**(p3 + p4*np.log(x/13000.)))*xwidths[mask[i]]/100 for i, x in enumerate(xdata[mask])])]) 
    
    elif (npar == 5):
        def signal_fit_func_array(parr):
            # see https://cds.cern.ch/record/2256663/files/B2G-17-001-pas.pdf
            # see the ATLAS dijet resonance search: https://arxiv.org/pdf/1806.00843.pdf.
            p1, p2, p3, p4, p5 = parr
            return np.array([np.sum([(p1*(1.-(x/13000.))**p2) / ((x/13000.)**(p3 + p4*np.log(x/13000.) + p5*(np.log(x/13000.))**2))*xwidths[mask[i]]/100 for i, x in enumerate(xdata[mask])])])
    
    else:
        print('Wrong number of parameters for the fit function')
    
    
    #Get covariance matrix of prediction uncertainties in the signal region
    jac=numdifftools.core.Jacobian(signal_fit_func_array)
    x_signal_cov=np.dot(np.dot(jac(popt),pcov),jac(popt).T)
    #Inverse signal region covariance matrix:
    inv_x_signal_cov = inv(x_signal_cov)
    
    #Get observed and predicted event counts in the signal region
    obs = np.array([np.sum(np.array(ydata)[mask]*np.array(xwidths)[mask]/100)])
    
    if (npar == 3):
        expected = np.array([np.sum([fit_func(xdata[targetbin],popt[0],popt[1],popt[2])*xwidths[targetbin]/100 for targetbin in mask])])
    elif (npar == 4):
        expected = np.array([np.sum([fit_func(xdata[targetbin],popt[0],popt[1],popt[2],popt[3])*xwidths[targetbin]/100 for targetbin in mask])])
    elif (npar == 5):
        expected = np.array([np.sum([fit_func(xdata[targetbin],popt[0],popt[1],popt[2],popt[3],popt[4])*xwidths[targetbin]/100 for targetbin in mask])])    
    else:
        print('Wrong number of parameters for the fit function')
        
    #Negative numerator of log likelihood ratio, for signal rate mu = 0
    def min_log_numerator(expected_nuis_arr):
        #expected_nuis_arr is the array of systematic background uncertainty nuisance parameters
        #These are event rate densities
        expected_nuis_arr = np.array(expected_nuis_arr)
        to_return = 0
        #Poisson terms
        for i, expected_nuis in enumerate(expected_nuis_arr):
            #Poisson lambda. Have to rescale nuisance constribution by bin width
            my_lambda = expected[i]+expected_nuis_arr[i]
            #Prevent negative predicted rates
            if my_lambda < 10**-10:
                my_lambda = 10**-10
            #Poisson term. Ignore the factorial piece which will cancel in likelihood ratio
            to_return = to_return + (obs[i]*np.log(my_lambda) - my_lambda)
            
        #Gaussian nuisance term
        nuisance_term = -0.5*np.dot(np.dot(expected_nuis_arr,inv_x_signal_cov),expected_nuis_arr)
        to_return = to_return + nuisance_term
        return -to_return

    def jac_min_log_numerator(expected_nuis_arr):
        #expected_nuis_arr is the array of systematic background uncertainty nuisance parameters
        #These are event rate densities
        expected_nuis_arr = np.array(expected_nuis_arr)
        to_return = np.array([0.])
        #Poisson terms
        #Poisson lambda. Have to rescale nuisance constribution by bin width
        my_lambda = expected+expected_nuis_arr
        dmy_lambda = np.array([1.])
        #Prevent negative predicted rates
        my_lambda[my_lambda < 10**-10] = np.ones(len(my_lambda[my_lambda < 10**-10])) * 10**-10
        dmy_lambda[my_lambda < 10**-10] = 0
        #Poisson term. Ignore the factorial piece which will cancel in likelihood ratio
        to_return = to_return + (obs*dmy_lambda/my_lambda - dmy_lambda)
        #Gaussian nuisance term
        nuisance_term = -np.dot(inv_x_signal_cov,expected_nuis_arr)
        to_return = to_return + nuisance_term
        return -to_return
    
    #Initialization of nuisance params
    expected_nuis_array_init = [0.02]
    
    #shift log likelihood to help minimization algo
    def rescaled_min_log_numerator(expected_nuis_arr):
        return min_log_numerator(expected_nuis_arr) - min_log_numerator(expected_nuis_array_init)
    
    #Perform minimization over nuisance parameters. Set bounds for bg nuisance at around 8 sigma.
    bnds=[[-8*y_unc[mask[0]],16*y_unc[mask[0]]]]
    minimize_log_numerator = minimize(rescaled_min_log_numerator,
                                      expected_nuis_array_init,
                                      jac=jac_min_log_numerator,
                                      bounds=bnds)
    
    if verbose:
        print("numerator: ",  minimize_log_numerator.items(),'\n')
        
    #Now get likelihood ratio denominator
    def min_log_denom(nuis_arr):
        #nuis_arr contains the bg systematics and also the signal rate
        expected_nuis_arr = np.array(nuis_arr)[:1]
        #print(expected_nuis_arr)
        mu = nuis_arr[1]
        #Signal prediction
        pred = [mu]
        to_return = 0
        #Poisson terms
        for i, expected_nuis in enumerate(expected_nuis_arr):
            #Poisson lambda
            my_lambda = expected[i]+expected_nuis_arr[i] + pred[i]
            #Prevent prediction from going negative
            if my_lambda < 10**-10:
                my_lambda = 10**-10
            #Poisson term. Ignore the factorial piece which will cancel in likelihood ratio
            to_return = to_return + (obs[i]*np.log(my_lambda) - my_lambda)

        #Gaussian nuisance term
        nuisance_term = -0.5*np.dot(np.dot(expected_nuis_arr,inv_x_signal_cov),expected_nuis_arr)
        to_return = to_return + nuisance_term
        return -to_return

    def jac_min_log_denom(nuis_arr):
        #expected_nuis_arr is the array of systematic background uncertainty nuisance parameters
        #These are event rate densities
        expected_nuis_arr = np.array(nuis_arr)[:1]
        mu = nuis_arr[1]
        pred = [mu]
        to_return_first = np.array([0.])
        #Poisson terms
        #Poisson lambda. Have to rescale nuisance constribution by bin width
        my_lambda = expected+expected_nuis_arr+pred
        dmy_lambda = np.array([1.])
        #Prevent prediction from going negative
        my_lambda[my_lambda < 10**-10] = np.ones(len(my_lambda[my_lambda < 10**-10])) * 10**-10
        dmy_lambda[my_lambda < 10**-10] = 0
        #Poisson term. Ignore the factorial piece which will cancel in likelihood ratio
        to_return_first = to_return_first + (obs*dmy_lambda/my_lambda - dmy_lambda)
        #Gaussian nuisance term
        nuisance_term = -np.dot(inv_x_signal_cov,expected_nuis_arr)
        to_return_first = to_return_first + nuisance_term
        
        to_return_last = np.array([0.])
        
        dpred = np.array([[1.]])
        
        my_lambda = expected+expected_nuis_arr+pred
        dmy_lambda = dpred
        to_return_last = np.dot((obs/my_lambda),dmy_lambda.T) - np.sum(dmy_lambda,axis=1)
        
        return -np.append(to_return_first, to_return_last)
    
    #initizalization for minimization
    nuis_array_init = [0.01,1.]
    
    #Shift log likelihood for helping minimization algo.
    def rescaled_min_log_denom(nuis_arr):
        return min_log_denom(nuis_arr) - min_log_denom(nuis_array_init)
    
    bnds = ((None,None),(None,None))
    minimize_log_denominator = minimize(rescaled_min_log_denom,nuis_array_init,
                                        jac=jac_min_log_denom,
                                        bounds=bnds)
    
    if verbose:
        print("Denominator: ",  minimize_log_denominator.items(),'\n')
        
    if minimize_log_denominator.x[-1] < 0:
        Zval = 0
        neglognum = 0
        neglogden = 0
    else:
        neglognum = min_log_numerator(minimize_log_numerator.x)
        neglogden = min_log_denom(minimize_log_denominator.x)
#        Zval = np.sqrt(2*(neglognum - neglogden))
        
#        print(neglognum - neglogden)
        if ((neglognum - neglogden) >= 0):
            Zval = np.sqrt(2*(neglognum - neglogden))
        elif ((neglognum - neglogden) > -0.25):
            Zval = np.sqrt(2*np.abs(neglognum - neglogden))
        else:
            Zval = np.sqrt(2*np.abs(neglognum - neglogden))
            print('Sqrt of potentially large negative number, fit might not be reliable')
      
    
    p0 = 1-norm.cdf(Zval)
    
    if verbose:
        print("z = ", Zval)
        print("p0 = ", p0)

    if (make_plot == True):
        plt.title('SR masked out | p0 = %.1e' % p0)
        plt.show()

    if return_teststat:
        return p0, 2*(neglognum - neglogden)
    else:
        return p0
    